In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib widget
import matplotlib
matplotlib.rc('font', size=18)

import pandas as pd
import xarray as xr

import sys
sys.path.append('../')

from general_purpose import cartopy_plots as cplt

In [ ]:
ts = xr.open_dataset('HEAVY--Land_and_Ocean_LatLong1.nc')

In [ ]:
ts

In [ ]:
LON,LAT = np.meshgrid(ts.longitude.data,ts.latitude.data)

## Plot the land_mask

In [ ]:
cplt.mfp(LON, LAT, ts.land_mask.data.reshape(*ts.land_mask.shape,1),
         figsize=(12,6),
         # draw_coastlines=False,
         projections=cplt.ccrs.PlateCarree(),
         colorbar='shared',
         extents=None,
         titles=['Annual mean temperature [C]'],
        )

## Plot the climatology

In [ ]:
ts.climatology.shape

In [ ]:
months=['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
ofl = (4,3)
ims = cplt.mfp(LON, LAT, ts.climatology.data.transpose(1,2,0),
               figsize=(16,9),
               one_fig_layout=ofl,
               # draw_coastlines=False,
               projections=cplt.ccrs.PlateCarree(),
               colorbar='shared',
               put_colorbar=False,
               extents=None,
               titles=months,
              )

plt.close(3)
fig,ax = plt.subplots(num=3, figsize=(10,2))
plt.colorbar(ims[0],cax=ax,orientation='horizontal', extend='both', label='T [C]')
fig.tight_layout()

### Compute climatology anomaly

In [ ]:
mean = ts.climatology.mean('month_number')

In [ ]:
cplt.mfp(LON, LAT, mean.data.reshape(*mean.shape,1),
         figsize=(12,6),
         # draw_coastlines=False,
         projections=cplt.ccrs.PlateCarree(),
         colorbar='shared',
         extents=None,
         titles=['Annual mean temperature [C]'],
         draw_labels=True,
         draw_gridlines=True
        )

In [ ]:
ano_clim = ts.climatology - mean

In [ ]:
months=['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
ofl = (4,3)
cplt.mfp(LON, LAT, ano_clim.data.transpose(1,2,0),
         figsize=(16,9),
         one_fig_layout=ofl,
         # draw_coastlines=False,
         projections=cplt.ccrs.PlateCarree(),
         colorbar='shared',
         extents=None,
         titles=months,
        )

## Plot a few snapshots

In [ ]:
ts.temperature.shape

In [ ]:
i = 12*100
ofl = (4,3)
ims = cplt.mfp(LON, LAT, ts.temperature.data[i:i+np.prod(ofl)].transpose(1,2,0),
               figsize=(16,9),
               one_fig_layout=ofl,
               # draw_coastlines=False,
               projections=cplt.ccrs.PlateCarree(),
               colorbar='shared',
               put_colorbar=False,
               extents=None,
               titles=months,
              )

plt.close(3)
fig,ax = plt.subplots(num=3, figsize=(10,2))
plt.colorbar(ims[0],cax=ax,orientation='horizontal', extend='both', label='T [C]')
fig.tight_layout()

In [ ]:
plt.close(19)
plt.figure(figsize=(9,6), num=19)
ts.temperature.isel(latitude=125, longitude=200).plot()
plt.plot(ts.time.data, ts.temperature.isel(longitude=200, latitude=125).data - ano_clim_periodic[:,125,200])
fig.tight_layout()

In [ ]:
pn = ts.temperature.isnull().mean(['longitude','latitude'])

In [ ]:
ocean_mask = ts.land_mask == 0
pno = ts.temperature.where(ocean_mask).isnull().mean(['longitude','latitude']) - (1 - np.mean(ocean_mask))

In [ ]:
atlantic_mask = ocean_mask.copy()

# mask out everything below 35 deg south
atlantic_mask[atlantic_mask.latitude < -35] = False
atlantic_mask[atlantic_mask.latitude > 80] = False

atlantic_mask[:,atlantic_mask.longitude > 25] = False
atlantic_mask[:,atlantic_mask.longitude < -120] = False

for i in range(atlantic_mask.shape[0]):
    if atlantic_mask.latitude[i] > 20 or atlantic_mask.latitude[i] < -35:
        continue
    j = np.argmax(atlantic_mask.data[i])
    while atlantic_mask.data[i,j]:
        j += 1
    atlantic_mask[i,:j] = False
    
atlantic_mask[:, atlantic_mask.longitude < -100] = False

plt.close(2)
fig = plt.figure(num=2, figsize=(12,9))
ax = fig.add_subplot(111, projection=cplt.ccrs.PlateCarree())

cplt.geo_plotter(ax, LON, LAT,
                 atlantic_mask,
                 greenwich=True,
                 draw_gridlines=False, draw_labels=False,
                 put_colorbar=True
                )

In [ ]:
pna = ts.temperature.where(atlantic_mask).isnull().mean(['longitude','latitude']) - (1 - np.mean(atlantic_mask))

In [ ]:
plt.close(4)
fig,ax = plt.subplots(figsize=(9,6), num=4)
pn.plot(label='all')
pno.plot(label='ocean only')
pna.plot(label='altantic only')
plt.ylabel('Fraction of missing points')
plt.xlabel('Year')
plt.legend()